<h1 style="font-size:2.1em;
           font-family: Calibri
           padding: 20px;
           margin: 20px;
           height: 100px;
           line-height: 100px;
           align-items: center;
           text-align: center;
           border: 3px solid #006400;
           background-color: #f1f1f1;
           "> Analysing the Tweets </h1>

In [1]:
#import packages
import quandl
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import nltk

from nltk.util import ngrams
from bs4 import BeautifulSoup


from datetime import datetime, timedelta

In [3]:
UALTweets = pd.read_csv("./Data/CompanyTweets/Tweets UAL 20200219 to 20200319.csv")
UALTweets['Tokens'] = UALTweets.Text.apply(lambda x: [x for x in ngrams(x.strip().split(' '),1)])
UALTweets['Bigrams'] = UALTweets.Text.apply(lambda x: [x for x in ngrams(x.strip().split(' '),2)])
UALTweets.head()

,Created,Text,User,Company,Symbol,TwitterTerm,Tokens,Bigrams
0,2020-03-18,Yo @united my flight from Punta Cana to Chicag...,1THC_Jake,United Airlines Holdings Inc.,UAL,@united,"[(Yo,), (@united,), (my,), (flight,), (from,),...","[(Yo, @united), (@united, my), (my, flight), (..."
1,2020-03-18,Heya @united Flown with you - a lot - for year...,SarahASimon,United Airlines Holdings Inc.,UAL,@united,"[(Heya,), (@united,), (Flown,), (with,), (you,...","[(Heya, @united), (@united, Flown), (Flown, wi..."
2,2020-03-18,We’re being told @united flight 4693 to DEN is...,GUC_Airport,United Airlines Holdings Inc.,UAL,@united,"[(We’re,), (being,), (told,), (@united,), (fli...","[(We’re, being), (being, told), (told, @united..."
3,2020-03-18,Can someone @CBP @CBPFlorida @united please te...,StGeorge_Abroad,United Airlines Holdings Inc.,UAL,@united,"[(Can,), (someone,), (@CBP,), (@CBPFlorida,), ...","[(Can, someone), (someone, @CBP), (@CBP, @CBPF..."
4,2020-03-18,Pretty convenient @united changed their itiner...,bdplayer81,United Airlines Holdings Inc.,UAL,@united,"[(Pretty,), (convenient,), (@united,), (change...","[(Pretty, convenient), (convenient, @united), ..."


### Most Popular Words

In [4]:
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\s.khanbhai\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
text = "Nick likes to play football, however he is not too fond of tennis."
text_tokens = ngrams(text.strip().split(' '),1)

tokens_without_sw = [word for word in text_tokens if not word in stopwords.words()]
print(tokens_without_sw)

[('Nick',), ('likes',), ('to',), ('play',), ('football,',), ('however',), ('he',), ('is',), ('not',), ('too',), ('fond',), ('of',), ('tennis.',)]


In [6]:
df = pd.DataFrame(UALTweets.Text.str.split(expand=True).stack().value_counts(),columns=['Count'])
df[~df.index.isin(stopwords.words())].head(50)

,Count
@united,5397
I,3411
$UAL,2051
flight,1815
@United,1101
@Delta,1055
United,1013
-,901
get,867
&amp;,784


In [7]:
df = pd.DataFrame([
    'must watch. Good acting',
    'average movie. Bad acting',
    'good movie. Good acting',
    'pathetic. Avoid',
    'avoid'], columns=['description'])
df['description']

0      must watch. Good acting
1    average movie. Bad acting
2      good movie. Good acting
3              pathetic. Avoid
4                        avoid
Name: description, dtype: object

In [8]:
UALTweets['Text'].head()

0    Yo @united my flight from Punta Cana to Chicag...
1    Heya @united Flown with you - a lot - for year...
2    We’re being told @united flight 4693 to DEN is...
3    Can someone @CBP @CBPFlorida @united please te...
4    Pretty convenient @united changed their itiner...
Name: Text, dtype: object

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
word_vectorizer = CountVectorizer(ngram_range=(2,2), analyzer='word')
sparse_matrix = word_vectorizer.fit_transform(UALTweets['Text'])
frequencies = sum(sparse_matrix).toarray()[0]
df = pd.DataFrame(frequencies, index=word_vectorizer.get_feature_names(), columns=['frequency'])\
        .sort_values(by='frequency', ascending=False)
df.head(40)

,frequency
https co,7144
united airlines,588
in the,583
united delta,581
americanair united,581
with united,525
on the,485
thank you,483
of the,481
at no,464


In [10]:
df.sort_values(by='frequency', ascending=False).head(30)

,frequency
https co,7144
united airlines,588
in the,583
americanair united,581
united delta,581
with united,525
on the,485
thank you,483
of the,481
at no,464


In [6]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [8]:
SentimentIntensityAnalyzer().polarity_scores("This Sucks")

{'compound': -0.3612, 'neg': 0.714, 'neu': 0.286, 'pos': 0.0}